# Convert a R script into Python

In [ ]:
rscript = """
nb=function(y=1930){
debut=1816
MatDFemale=matrix(D$Female,nrow=111)
colnames(MatDFemale)=debut+0:198
cly=(y-debut+1):111
deces=diag(MatDFemale[:,cly[cly%in%1:199]])
return(c(B$Female[B$Year==y],deces))}
"""

In [ ]:
from pyensae.languages import r2python
print(r2python(rscript, pep8=True))


def nb(y=1930):

    debut = 1816
    MatDFemale = matrix(D . Female, nrow=111)
    colnames(MatDFemale) .set(debut + 0, 198)
    cly = (y - debut + 1), 111
    deces = diag(MatDFemale[:, cly[cly in 1, 199]])
    return tuple(B . Female[B . Year == y], deces)



In [ ]:
[4,5] & [4]

TypeError: unsupported operand type(s) for &: 'list' and 'list'